In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
import torch

In [3]:
from pytorch_lightning.core.lightning import LightningModule

In [4]:
from recibrew.core_nn import TransformersLightning

In [5]:
tl = TransformersLightning()

In [6]:
tl.hparams

"dev_csv":           ../data/processed/dev.csv
"dim_feedforward":   512
"dropout":           0.3
"lr":                0.001
"num_decoder_layer": 4
"num_embedding":     128
"num_encoder_layer": 4
"padding_idx":       1
"test_csv":          ../data/processed/test.csv
"train_csv":         ../data/processed/train.csv

In [7]:
from pytorch_lightning import Trainer, seed_everything

In [8]:
from pytorch_lightning.callbacks import EarlyStopping

In [9]:
seed_everything(888)

888

In [10]:
tl.summarize(mode='top')


  | Name             | Type            | Params
-----------------------------------------------------
0 | full_transformer | FullTransformer | 2 M   


  | Name             | Type            | Params
-----------------------------------------------------
0 | full_transformer | FullTransformer | 2 M   

In [11]:
early_stop_callback = EarlyStopping(
   monitor='val_loss',
   min_delta=0.00,
   patience=3,
   verbose=False,
   mode='auto'
)
trainer = Trainer(early_stop_callback=early_stop_callback, accumulate_grad_batches=2, auto_lr_find=False, gpus=[0])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(tl)


  | Name             | Type            | Params
-----------------------------------------------------
0 | full_transformer | FullTransformer | 2 M   


C:\Anaconda\envs\recibrew\lib\site-packages\pytorch_lightning\utilities\distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [15]:
tl.hparams

"dev_csv":           ../data/processed/dev.csv
"dim_feedforward":   512
"dropout":           0.3
"lr":                0.001
"num_decoder_layer": 4
"num_embedding":     128
"num_encoder_layer": 4
"padding_idx":       1
"test_csv":          ../data/processed/test.csv
"train_csv":         ../data/processed/train.csv

In [16]:
src_field = tl.constructed_iterator_field['src_field']

In [17]:
tgt_field = tl.constructed_iterator_field['tgt_field']

In [53]:
src_input = [src_field.vocab[x] for x in '<s> kambing guling </s>'.split()]

In [54]:
tl = tl.eval()

In [55]:
src_input

[2, 57, 2556, 3]

Decode

In [56]:
src_tensor = torch.LongTensor(src_input).unsqueeze(1).cuda()

tgt_input = [tgt_field.vocab[x] for x in '<s>'.split()]

for i in range(130):
    trg_tensor = torch.LongTensor(tgt_input).unsqueeze(1).cuda()
    
    output = tl.forward(src_tensor, trg_tensor)
    out_token = output.argmax(2)[-1].item()
    tgt_input.append(out_token)
    if out_token == 3:
        break

Evaluate BLEU and FINISHED

In [57]:
' '.join([tgt_field.vocab.itos[x] for x in tgt_input])

'<s> 1 / 2 kg daging kambing || 1 / 2 kg daging kambing || 1 / 2 kg daging kambing || 1 / 2 sdt garam || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt gula pasir || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 / 2 sdt merica bubuk || 1 sdt gula || 1 / 2 sdt garam || 1 / 2 sdt gula || 1 / 2'